## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

In [19]:
import mne
import numpy as np

file_path = "datasets/avi/single/sub1_combined-raw.fif"

epochs = mne.read_epochs(file_path, preload=True)

threshold = 0.25



psd_values = []

targets = [float(item) for item in epochs.event_id.keys()]

features = list()
## passo 3: computar PSD para cada frequencia


for i in range(len(epochs)):
    sample = list()
    for freq in targets:
        # filtra os dados para a faixa de frequencia 
        fmin = freq - threshold
        fmax = freq + threshold
        sample.append(epochs[i].compute_psd(fmin=fmin, fmax=fmax,verbose=False)[0][0])
    features.append(sample)

X = np.array(features)
print(X.shape)
# passo 4: extrair recursos dos valores PSD
X = X.reshape((X.shape[0], X.shape[1], X.shape[-1]))

max_values = np.max(X, axis=-1)
max_values.shape

#passo 5: classificar os vetores de recursos
file_path = "datasets/avi/single/labels_sub1.npy"
y = np.load(file_path).flatten()
print(y, y.shape)


i_max = np.argmax(max_values, axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print("Acuracia: {}%".format(acc))






Reading c:\Users\caio-\Documents\facul\patter-recog\SSVEP\datasets\avi\single\sub1_combined-raw.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   29998.05 ms
        0 CTF compensation matrices available
Not setting metadata
84 matching events found
No baseline correction applied
0 projection items activated


C:\Users\caio-\AppData\Local\Temp\ipykernel_9380\1878604750.py:6: RuntimeWarning: This filename (datasets/avi/single/sub1_combined-raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(file_path, preload=True)


(84, 7, 1, 1, 15)
[10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3
 10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3] (84,)
Acuracia: 85.71428571428571%


In [36]:
file_path = "datasets/avi/multi/sub1_combined-raw.fif"

epochs = mne.read_epochs(file_path, preload=True)
threshold = 0.22

targets = [float(item) for item in epochs.event_id.keys()]
print("Possíveis alvos:", targets)

features = list()


## passo 3: computar PSD para cada frequencia

for i in range(len(epochs)):
    sample = list()
    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        psd = epochs[i].compute_psd(fmin=fmin, fmax=fmax, method='multitaper', verbose=False)
       
        avg_psd = np.mean(psd, axis=(0, 2))
        sample.append(avg_psd)
    features.append(sample)


print(features)
X = np.array(features)


# passo 4: extrair recursos dos valores PSD
X = X.reshape(X.shape[0], X.shape[1], X.shape[-1])

max_values = np.max(X, axis=-1)
max_values.shape


#passo 5: classificar os vetores de recursos
y = np.load("datasets/avi/multi/labels.npy")

print(y, y.shape, targets)

i_max = max_values.argmax(axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print(f'\nPorcentagem de acerto: {acc:.2f}%')

Reading c:\Users\caio-\Documents\facul\patter-recog\SSVEP\datasets\avi\multi\sub1_combined-raw.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   15998.05 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated


C:\Users\caio-\AppData\Local\Temp\ipykernel_9380\2366082550.py:3: RuntimeWarning: This filename (datasets/avi/multi/sub1_combined-raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(file_path, preload=True)


Possíveis alvos: [6.0, 6.5, 7.0, 7.5, 8.2, 9.3]
(100, 1, 8192)
[[array([1.1178052e-13]), array([1.12980102e-13]), array([7.32827884e-14]), array([6.86633335e-14]), array([5.27288481e-14]), array([4.16861107e-14])], [array([9.32291192e-14]), array([4.12354286e-14]), array([2.37511648e-14]), array([2.55264483e-14]), array([1.32305009e-14]), array([1.09922272e-14])], [array([6.36277797e-14]), array([3.37586689e-14]), array([1.97554232e-14]), array([1.93213479e-14]), array([1.67122958e-14]), array([1.85379494e-14])], [array([8.56973644e-14]), array([6.34650119e-14]), array([5.4230604e-14]), array([4.0468772e-14]), array([3.70689711e-14]), array([3.83915855e-14])], [array([3.88790036e-14]), array([2.24382093e-14]), array([2.24457246e-14]), array([1.5781771e-14]), array([1.46054921e-14]), array([9.7379978e-15])], [array([6.68503002e-14]), array([6.92409557e-14]), array([5.28482081e-14]), array([3.90839475e-14]), array([3.29165866e-14]), array([2.54911459e-14])], [array([6.65483412e-14]), arr